# multi_columnsの動作確認(2行ヘッダー版)

In [2]:
from IPython.display import display
import pandas as pd
import sys
from os.path import abspath, split, join, dirname, realpath
from inspect import getfile, currentframe
pjc_folder = realpath(dirname(abspath(split(getfile( currentframe() ))[0])))

if pjc_folder not in sys.path:
    sys.path.insert(0, pjc_folder)
pass

In [3]:
# これは実行上必要なものではないです(GitHubにあげる上で、ファイルパスなどを消すためにワーニングを非表示)
import warnings
warnings.filterwarnings('ignore')

In [4]:
xl_file = 'test_col2.xlsx'
xi_path = join(pjc_folder, 'file' , xl_file)

In [5]:
#まずは普通にpandasのread_excelを使って読む
df = pd.read_excel(xi_path, skiprows=2, header=[0,1])
display(df)

No      状態                       備考
        体重  体脂肪率 Unnamed: 2_level_1
1001  65.5  19.5                NaN
1002  66.0  18.1               飲み過ぎ
1003  65.7  19.1                NaN
1004  65.8  19.3                NaN
1005  65.3  20.1                NaN

In [6]:
# 次に作成したプログラムを読み込んで、multi_columns関数に食わせる
from convert_excel import multi_columns
df = multi_columns(df)
display(df)

No    状態          備考
           体重  体脂肪率      
0  1001  65.5  19.5   NaN
1  1002  66.0  18.1  飲み過ぎ
2  1003  65.7  19.1   NaN
3  1004  65.8  19.3   NaN
4  1005  65.3  20.1   NaN

In [7]:
df.columns.names

FrozenList([None, None])

In [8]:
list(df.columns)

[('No', ''), ('状態', '体重'), ('状態', '体脂肪率'), ('備考', '')]

### ここから色々操作してみる

In [9]:
# まずは子の項目のないものから
df["備考"]

0     NaN
1    飲み過ぎ
2     NaN
3     NaN
4     NaN
Name: 備考, dtype: object

In [10]:
# Noは作られた直後はindexではない。普通の列項目
df["No"]

0    1001
1    1002
2    1003
3    1004
4    1005
Name: No, dtype: int64

In [11]:
# 次に階層化してあるもの(複数キーがないと断定できないもの)
df["状態", "体重"]

0    65.5
1    66.0
2    65.7
3    65.8
4    65.3
Name: (状態, 体重), dtype: float64

In [12]:
# 状態だけを選ぶと子の要素二つが選ばれる
df["状態"]

,体重,体脂肪率
0,65.5,19.5
1,66.0,18.1
2,65.7,19.1
3,65.8,19.3
4,65.3,20.1


In [13]:
# No列をインデックスにする場合は以下の通りにすればよい。パフォーマンス上の警告が出るが実施可能。
df.index = df["No"]
df.index.name = "No"
df = df.drop(columns="No")
display(df)

状態          備考
        体重  体脂肪率      
No                    
1001  65.5  19.5   NaN
1002  66.0  18.1  飲み過ぎ
1003  65.7  19.1   NaN
1004  65.8  19.3   NaN
1005  65.3  20.1   NaN

In [14]:
# 列を選択すると、Noがキーとして出力されている
df["備考"]

No
1001     NaN
1002    飲み過ぎ
1003     NaN
1004     NaN
1005     NaN
Name: 備考, dtype: object